In [1]:
from datetime import datetime, timedelta
import re
import sqlite3
import time
import pandas as pd
pd.set_option('display.max_columns', None)
from bs4 import BeautifulSoup as bs4
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime

In [28]:
# Create an Engine object
engine = create_engine('sqlite:///fantasy_data.db', echo=False)

chromedriver_path = "C:\\ProgramData\\Anaconda3\\WebDriver\\bin\\chromedriver.exe"
url = 'https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin'

draft_date = datetime(2025, 3, 15)
days_since_draft = (datetime.now() - draft_date).days

In [38]:
team_list = ['Lima Time', 'Ugly Spuds', 'Harveys Wallbangers', 'Roid Ragers', 'Charmer', 'Wiscompton Wu-Tang', '9 Grand Kids', 'Brewbirds', 'Trouble With The Curve', "Mom's Cookin", 'Dirty Birds', 'Young Guns']
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'Sv+Hld', 'SO', 'QS', 'WHIP']

owner_dict = {"9 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45'}
position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']

name_change = {"9 Gran...":'9 Grand Kids', 'Brewbi...':'Brewbirds', 'Charme...':'Charmer', 'Dirty ...':'Dirty Birds', 
               'Harvey...':'Harveys Wallbangers', 'Lil Tr...':'Lil Trump & the Ivanabees', 'Lima T...':'Lima Time!', 
               'Roid R...':'Roid Ragers', 'Troubl...':'Trouble with the Curve', 'Ugly S...':'Ugly Spuds', 
               'Wiscom...':'Wiscompton Wu-Tang', 'Young ...':'Young Guns', "Mom's ...":"Mom's Cookin"}

In [37]:
year = datetime.now().year
opening_week = int(datetime.strftime(pd.Timestamp('2024-03-18'), '%W')) - 1
cur_week = int(datetime.strftime(datetime.now(), '%W'))
period = cur_week - opening_week
current_period = period + 1
opening_week, cur_week, period, current_period

(11, 11, 0, 1)

In [3]:
from scraper import Scraper
s = Scraper()

In [4]:
s.get_cbs_projections('h')

Waiting for projections page to load...
Stats type: h
Accessing URL: https://xdl.baseball.cbssports.com/stats/stats-main/all:C:1B:2B:3B:SS:MI:CI:OF:DH/restofseason:p/standard/projections?print_rows=9999
Page loaded at least 2500 rows


C:\Users\pddnh\Documents\GitHub\xdl\scraper.py:145: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=1, skiprows=1, extract_links='body')[0]


ValueError: invalid literal for int() with base 10: 'h'

### Manual approach

In [5]:
# if that doesn't work
h_proj_url = 'https://xdl.baseball.cbssports.com/stats/stats-main/all:C:1B:2B:3B:SS:MI:CI:OF:DH/restofseason:p/standard/projections?print_rows=9999'
p_proj_url = 'https://xdl.baseball.cbssports.com/stats/stats-main/all:SP:RP:P/restofseason:p/standard/projections?print_rows=9999'

In [6]:
driver = s.cbs_login()

In [7]:
stats_type = 'h'
#stats_type = 'p'

In [8]:
if stats_type == 'h':
    driver.get(h_proj_url)
else:
    driver.get(p_proj_url)

In [9]:
#time.sleep(10)
html = driver.page_source
soup = bs4(html, 'html.parser')

table = soup.find('table', {'class': 'data'})

In [18]:
df = pd.read_html(str(table), header=1, skiprows=1, extract_links='body')[0]
df = df.iloc[:-1, 2:]
df.head()

C:\Users\pddnh\AppData\Local\Temp\ipykernel_31892\3008177304.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=1, skiprows=1, extract_links='body')[0]


,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank
0,"(Shohei Ohtani DH,SP • LAD, /players/playerpag...","(546, None)","(125, None)","(172, None)","(85, None)","(37, None)","(6, None)","(44, None)","(117, None)","(72, None)","(137, None)","(45, None)","(5, None)","(.315, None)","(.402, None)","(.646, None)","(1, None)"
1,"(Aaron Judge OF • NYY, /players/playerpage/207...","(521, None)","(112, None)","(162, None)","(76, None)","(35, None)","(2, None)","(49, None)","(126, None)","(106, None)","(177, None)","(8, None)","(1, None)","(.311, None)","(.434, None)","(.668, None)","(2, None)"
2,"(Bobby Witt SS • KC, /players/playerpage/3117472)","(578, None)","(112, None)","(195, None)","(114, None)","(42, None)","(8, None)","(31, None)","(100, None)","(56, None)","(117, None)","(28, None)","(11, None)","(.337, None)","(.400, None)","(.599, None)","(3, None)"
3,"(Jose Ramirez 3B • CLE, /players/playerpage/20...","(553, None)","(102, None)","(161, None)","(85, None)","(36, None)","(4, None)","(36, None)","(106, None)","(61, None)","(80, None)","(34, None)","(6, None)","(.291, None)","(.361, None)","(.566, None)","(4, None)"
4,"(Francisco Lindor SS • NYM, /players/playerpag...","(582, None)","(111, None)","(164, None)","(90, None)","(37, None)","(4, None)","(33, None)","(96, None)","(63, None)","(130, None)","(28, None)","(4, None)","(.282, None)","(.358, None)","(.529, None)","(5, None)"


In [19]:
# Apply a lambda function to each column to extract the first element of each tuple
df = df.apply(lambda col: [v[0] if v[1] is None else v for v in col])
df.head()

,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank
0,"(Shohei Ohtani DH,SP • LAD, /players/playerpag...",546,125,172,85,37,6,44,117,72,137,45,5,.315,.402,.646,1
1,"(Aaron Judge OF • NYY, /players/playerpage/207...",521,112,162,76,35,2,49,126,106,177,8,1,.311,.434,.668,2
2,"(Bobby Witt SS • KC, /players/playerpage/3117472)",578,112,195,114,42,8,31,100,56,117,28,11,.337,.400,.599,3
3,"(Jose Ramirez 3B • CLE, /players/playerpage/20...",553,102,161,85,36,4,36,106,61,80,34,6,.291,.361,.566,4
4,"(Francisco Lindor SS • NYM, /players/playerpag...",582,111,164,90,37,4,33,96,63,130,28,4,.282,.358,.529,5


In [13]:
pattern = r'^(?P<FullName>[A-Za-z\.\'\-\s]+) (?P<Positions>[A-Z0-9,]+) • (?P<Team>[A-Z]+)$'

In [20]:
# Extract the player name and ID
df['player'] = df['Player'].apply(lambda x: x[0])
df['id'] = df['Player'].apply(lambda x: x[1])
# Parse id for cbsid
df['cbsid'] = df['id'].apply(lambda x: int(x.replace('/players/playerpage/', '')))
# Use regex to parse out CBSNAME, Pos, and Team
df[['CBSNAME', 'Positions', 'Team']] = df['player'].str.extract(pattern)
df.head()

,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank,player,id,cbsid,CBSNAME,Positions,Team
0,"(Shohei Ohtani DH,SP • LAD, /players/playerpag...",546,125,172,85,37,6,44,117,72,137,45,5,.315,.402,.646,1,"Shohei Ohtani DH,SP • LAD",/players/playerpage/2901324,2901324,Shohei Ohtani,"DH,SP",LAD
1,"(Aaron Judge OF • NYY, /players/playerpage/207...",521,112,162,76,35,2,49,126,106,177,8,1,.311,.434,.668,2,Aaron Judge OF • NYY,/players/playerpage/2071264,2071264,Aaron Judge,OF,NYY
2,"(Bobby Witt SS • KC, /players/playerpage/3117472)",578,112,195,114,42,8,31,100,56,117,28,11,.337,.400,.599,3,Bobby Witt SS • KC,/players/playerpage/3117472,3117472,Bobby Witt,SS,KC
3,"(Jose Ramirez 3B • CLE, /players/playerpage/20...",553,102,161,85,36,4,36,106,61,80,34,6,.291,.361,.566,4,Jose Ramirez 3B • CLE,/players/playerpage/2044495,2044495,Jose Ramirez,3B,CLE
4,"(Francisco Lindor SS • NYM, /players/playerpag...",582,111,164,90,37,4,33,96,63,130,28,4,.282,.358,.529,5,Francisco Lindor SS • NYM,/players/playerpage/1894627,1894627,Francisco Lindor,SS,NYM


In [39]:
df

,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank,player,id,cbsid,CBSNAME,Positions,Team
0,"(Shohei Ohtani DH,SP • LAD, /players/playerpag...",546,125,172,85,37,6,44,117,72,137,45,5,.315,.402,.646,1,"Shohei Ohtani DH,SP • LAD",/players/playerpage/2901324,2901324,Shohei Ohtani,"DH,SP",LAD
1,"(Aaron Judge OF • NYY, /players/playerpage/207...",521,112,162,76,35,2,49,126,106,177,8,1,.311,.434,.668,2,Aaron Judge OF • NYY,/players/playerpage/2071264,2071264,Aaron Judge,OF,NYY
2,"(Bobby Witt SS • KC, /players/playerpage/3117472)",578,112,195,114,42,8,31,100,56,117,28,11,.337,.400,.599,3,Bobby Witt SS • KC,/players/playerpage/3117472,3117472,Bobby Witt,SS,KC
3,"(Jose Ramirez 3B • CLE, /players/playerpage/20...",553,102,161,85,36,4,36,106,61,80,34,6,.291,.361,.566,4,Jose Ramirez 3B • CLE,/players/playerpage/2044495,2044495,Jose Ramirez,3B,CLE
4,"(Francisco Lindor SS • NYM, /players/playerpag...",582,111,164,90,37,4,33,96,63,130,28,4,.282,.358,.529,5,Francisco Lindor SS • NYM,/players/playerpage/1894627,1894627,Francisco Lindor,SS,NYM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3940,"(Mason Janvrin OF • BAL, /players/playerpage/3...",0,0,0,0,0,0,0,0,0,0,0,0,.000,.000,.000,9999,Mason Janvrin OF • BAL,/players/playerpage/3157970,3157970,Mason Janvrin,OF,BAL
3941,"(Antonio Gomez C • NYY, /players/playerpage/26...",0,0,0,0,0,0,0,0,0,0,0,0,.000,.000,.000,9999,Antonio Gomez C • NYY,/players/playerpage/26912470,26912470,Antonio Gomez,C,NYY
3942,"(Wander Javier SS • MIN, /players/playerpage/2...",0,0,0,0,0,0,0,0,0,0,0,0,.000,.000,.000,9999,Wander Javier SS • MIN,/players/playerpage/2507352,2507352,Wander Javier,SS,MIN
3943,"(Bayron Lora OF • TEX, /players/playerpage/315...",0,0,0,0,0,0,0,0,0,0,0,0,.000,.000,.000,9999,Bayron Lora OF • TEX,/players/playerpage/3151951,3151951,Bayron Lora,OF,TEX


In [22]:
if stats_type=='h':
    # Be sure to convert all columns to appropriate data types
    for col in ['AB', 'R', 'H', '1B', '2B', '3B', 'HR', 'RBI', 'BB', 'K', 'SB', 'CS', 'Rank']:
        df[col] = df[col].astype(int)
    for col in ['AVG', 'OBP', 'SLG']:
        df[col] = df[col].astype(float)
    df = df[['cbsid', 'CBSNAME', 'Positions', 'Team', 'AB', 'R', 'H', '1B', '2B', '3B', 'HR', 'RBI', 'BB', 'K', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'Rank']]
    if datetime.now() > draft_date:
        df[df['AB']>=0].to_csv(f'{s.destination_path}/{datetime.now().year}-period-{period}-ros-projections-{stats_type}.csv', index=False)
        print(f'{datetime.now().year}-cbs-proj-{stats_type}.csv saved in {s.destination_path}')
    else:
        df[df['AB']>=0].to_csv(f'{s.destination_path}/{datetime.now().year}-cbs-proj-{stats_type}.csv', index=False)
        print(f'{datetime.now().year}-cbs-proj-{stats_type}.csv saved in {s.destination_path}')
if stats_type=='p':
    # Be sure to convert all columns to appropriate data types
    for col in ['INNs', 'APP', 'GS', 'QS', 'CG', 'W', 'L', 'S', 'BS', 'HD', 'K', 'BB', 'H', 'Rank']:
        df[col] = df[col].astype(int)
    for col in ['ERA', 'WHIP']:
        df[col] = df[col].astype(float)
    df = df[['cbsid', 'CBSNAME', 'Positions', 'Team', 'INNs', 'GS', 'QS', 'W', 'L', 'S', 'HD', 'K', 'BB', 'H', 'ERA', 'WHIP', 'Rank']]
    # Save to csv if projected IP > 0
    df[df['INNs']>0].to_csv(f'{s.destination_path}/{datetime.now().year}-cbs-proj-{stats_type}.csv', index=False)
    print(f'{datetime.now().year}-cbs-proj-{stats_type}.csv saved in {s.destination_path}')

2025-cbs-proj-p.csv saved in C:\Users\pddnh\Documents\GitHub\xdl\data


In [23]:
driver.quit()